In [1]:
import pandas as pd

In [2]:
# Read the CSV file into a DataFrame
df = pd.read_csv("data/raw/recipes_sample.csv")

In [3]:
# print all columns and verify if any missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 28 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   RecipeId                    5000 non-null   int64  
 1   Name                        5000 non-null   object 
 2   AuthorId                    5000 non-null   int64  
 3   AuthorName                  5000 non-null   object 
 4   CookTime                    4183 non-null   object 
 5   PrepTime                    5000 non-null   object 
 6   TotalTime                   5000 non-null   object 
 7   DatePublished               5000 non-null   object 
 8   Description                 5000 non-null   object 
 9   Images                      5000 non-null   object 
 10  RecipeCategory              4998 non-null   object 
 11  Keywords                    4846 non-null   object 
 12  RecipeIngredientQuantities  5000 non-null   object 
 13  RecipeIngredientParts       5000 

In [4]:
# Drop all columns that is not required for analysis
clean_df = df.drop(["AuthorName", "AuthorId", "DatePublished", "CookTime", "PrepTime"], axis=1)

In [5]:
# fill NA in rating with 0 for 0 review
clean_df['AggregatedRating'].fillna(0, inplace=True)
clean_df['ReviewCount'].fillna(0, inplace=True)

In [6]:
# Splitting the 'RecipeYield' column into two new columns
clean_df[['Serving', 'Unit']] = clean_df['RecipeYield'].str.split(' ', n=1, expand=True)

# Converting 'Serving' column to integer
clean_df['Serving'] = clean_df['Serving'].str.extract('(\d+)').astype(float)

# Dropping the original 'RecipeYield' column and the 'Unit' column
clean_df.drop(['RecipeYield', 'Unit'], axis=1, inplace=True)

In [7]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   RecipeId                    5000 non-null   int64  
 1   Name                        5000 non-null   object 
 2   TotalTime                   5000 non-null   object 
 3   Description                 5000 non-null   object 
 4   Images                      5000 non-null   object 
 5   RecipeCategory              4998 non-null   object 
 6   Keywords                    4846 non-null   object 
 7   RecipeIngredientQuantities  5000 non-null   object 
 8   RecipeIngredientParts       5000 non-null   object 
 9   AggregatedRating            5000 non-null   float64
 10  ReviewCount                 5000 non-null   float64
 11  Calories                    5000 non-null   float64
 12  FatContent                  5000 non-null   float64
 13  SaturatedFatContent         5000 

In [8]:
# Fill NA in "RecipeServings" with "Serving" value where "RecipeServings" is NA and "Serving" is not null
clean_df.loc[clean_df['RecipeServings'].isna() & clean_df['Serving'].notna(), 'RecipeServings'] = clean_df['Serving']

# Convert 'RecipeServings' column to integer type if necessary
clean_df['RecipeServings'] = clean_df['RecipeServings'].astype(float)

In [9]:
# Fill NA in "Keywords" with values from "RecipeIngredientParts"
clean_df['Keywords'].fillna(clean_df['RecipeIngredientParts'], inplace=True)

In [10]:
clean_df = clean_df.drop(["Serving"], axis=1)

In [11]:
# Fill NA in "RecipeServings" with 1
clean_df['RecipeServings'].fillna(1, inplace=True)

In [12]:
# Remove rows with NA in "RecipeCategory"
clean_df = clean_df.dropna(subset=['RecipeCategory', 'Description', 'RecipeIngredientQuantities'])
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4998 entries, 0 to 4999
Data columns (total 22 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   RecipeId                    4998 non-null   int64  
 1   Name                        4998 non-null   object 
 2   TotalTime                   4998 non-null   object 
 3   Description                 4998 non-null   object 
 4   Images                      4998 non-null   object 
 5   RecipeCategory              4998 non-null   object 
 6   Keywords                    4998 non-null   object 
 7   RecipeIngredientQuantities  4998 non-null   object 
 8   RecipeIngredientParts       4998 non-null   object 
 9   AggregatedRating            4998 non-null   float64
 10  ReviewCount                 4998 non-null   float64
 11  Calories                    4998 non-null   float64
 12  FatContent                  4998 non-null   float64
 13  SaturatedFatContent         4998 non-n

In [13]:
# clean_df.head()

# Function to extract hours and minutes from a duration string
def extract_hours_minutes(duration_str):
    # Remove "PT" prefix
    duration_str = duration_str.replace("PT", "")

    # Initialize hours and minutes
    hours = 0
    minutes = 0

    # Split into hours and minutes if present
    if 'H' in duration_str:
        hours, duration_str = duration_str.split('H')
        hours = int(hours)
    if 'M' in duration_str:
        minutes = duration_str.replace('M', '')
        minutes = int(minutes)

    return hours, minutes

# Apply the function to the "TotalTime" column
clean_df['TotalTime_hours'], clean_df['TotalTime_minutes'] = zip(*clean_df['TotalTime'].apply(extract_hours_minutes))
clean_df.head()

In [14]:
# Remove "PT" prefix
clean_df['TotalTime'] = clean_df['TotalTime'].str.replace("PT", "")
clean_df.head()

,RecipeId,Name,TotalTime,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,AggregatedRating,...,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeInstructions
0,429918,Easy Chicken and Biscuits,30M,This is an easy recipe that can be done in 30 ...,character(0),One Dish Meal,"""< 30 Mins""","c(""1"", ""1"", ""1"", ""1/4"", ""1/4"", ""4"", ""2"", ""1"")","c(""milk"", ""dried thyme leaves"", ""pepper"", ""ham"")",0.0,...,17.0,5.3,58.6,1644.3,37.0,1.5,6.5,20.8,5.0,"c(""In a 3-quart shallow baking dish mix soups,..."
1,498503,Spring Gnocchi With Asparagus and Shrimp,25M,A perfect blend of the heartiest of winter pas...,character(0),Weeknight,"c(""< 30 Mins"", ""Easy"")","c(""32 -36"", ""4"", ""1"", ""1"", ""1"", ""2"", ""1"", ""4"",...","c(""butter"", ""lemon, juice and zest of"", ""salt""...",0.0,...,11.0,6.4,121.2,1018.0,8.5,3.4,2.4,16.3,6.0,"c(""Boil a large pot of salted water and add gn..."
2,263162,Romano Grits,20M,"This recipe can be multiplied by 2, 3, 4. The...",character(0),Breakfast,"c(""Very Low Carbs"", ""Low Protein"", ""Low Choles...","c(""2"", ""1/2"", ""1/4"", ""1"")","c(""water"", ""yellow corn grits"", ""salt"", ""pecor...",5.0,...,3.9,2.4,14.7,466.0,3.7,0.1,0.1,4.9,2.0,"c(""Place the water in a medium sauce pan over ..."
3,72878,Lemon Chess Pie,50M,Make and share this Lemon Chess Pie recipe fro...,character(0),Pie,"c(""Dessert"", ""< 60 Mins"")","c(""1 1/2"", ""2"", ""4"", ""1"", ""1/2"", ""1"")","c(""sugar"", ""butter"", ""eggs"", ""fresh lemon rind"")",4.0,...,12.8,4.5,113.4,172.4,48.5,0.9,37.8,4.6,8.0,"c(""Pre heat oven to 400."", ""Cream together but..."
4,350007,Slow Cooked Chicken and Dressing,4H25M,This is so delicious! If you want an easy com...,character(0),One Dish Meal,"c(""Chicken"", ""Poultry"", ""Meat"", ""Kid Friendly""...","c(""1"", ""6"", ""8"", ""2"", ""2"", ""1"", ""3"", ""4"", ""2"",...","c(""chicken broth"", ""onion"", ""celery ribs"", ""eg...",0.0,...,27.6,11.8,191.3,1275.0,22.5,1.2,3.0,24.8,1.0,"c(""Combine first 11 ingredients in a large bow..."


In [15]:
# Create a new column "URL" with the initial value from "Name" and "RecipeId" column
# Food.com url example "https://www.food.com/recipe/japanese-pumpkin-soup-kabocha-soup-88935"

#Create new column URL to store generated URL Link
clean_df['URL'] = clean_df['Name']

# Remove parentheses, dashes, and commas from the "URL" column
clean_df['URL'] = clean_df['URL'].str.replace(r'[\(\)\-,]', '')  

# Select the "URL" column and apply transformations
clean_df['URL'] = clean_df['URL'].str.replace(' ', '-')  # Replace spaces with hyphens
clean_df['URL'] = clean_df['URL'].str.lower()  # Convert text to lowercase

# Concatenate "https://www.food.com/recipe/" with the modified "URL" column
clean_df['URL'] = 'https://www.food.com/recipe/' + clean_df['URL'] + '-' + clean_df['RecipeId'].astype(str)
clean_df['URL']

0       https://www.food.com/recipe/easy-chicken-and-b...
1       https://www.food.com/recipe/spring-gnocchi-wit...
2         https://www.food.com/recipe/romano-grits-263162
3       https://www.food.com/recipe/lemon-chess-pie-72878
4       https://www.food.com/recipe/slow-cooked-chicke...
                              ...                        
4995    https://www.food.com/recipe/simplified-cassoul...
4996    https://www.food.com/recipe/tomato-and-pepper-...
4997    https://www.food.com/recipe/bbq'd-pork-&-prune...
4998    https://www.food.com/recipe/homemade-buttermil...
4999       https://www.food.com/recipe/easy-quiche-500686
Name: URL, Length: 4998, dtype: object

In [16]:
import re

# Remove "c" at the beginning and parentheses from each value in the "RecipeIngredientParts" column
#clean_df['RecipeIngredientParts'] = clean_df['RecipeIngredientParts'].str.replace(r'^c|[()]', '', regex=True)

# Remove "c" at the beginning and parentheses from each value in the "RecipeIngredientParts" column
clean_df['Keywords'] = clean_df['Keywords'].str.replace(r'^c|[()]', '', regex=True)
clean_df['RecipeIngredientQuantities'] = clean_df['RecipeIngredientQuantities'].str.replace(r'^c|[()]', '', regex=True)
clean_df['RecipeIngredientParts'] = clean_df['RecipeIngredientParts'].str.replace(r'^c|[()]', '', regex=True)
clean_df['RecipeInstructions'] = clean_df['RecipeInstructions'].str.replace(r'^c|[()]', '', regex=True)
clean_df['Images'] = clean_df['Images'].str.replace(r'^c|[()]', '', regex=True)

In [17]:
# Split the strings by comma, explode the resulting lists, and get unique items
keyword_item = clean_df['Keywords'].str.split(',').explode().str.strip().unique()

# Convert to list
keyword_item_list = keyword_item.tolist()

# Print the length of the list
print("Number of unique items:", len(keyword_item_list))

Number of unique items: 574


In [18]:
# Clean up each string (remove leading/trailing spaces, convert to lowercase, and strip double quotes) before converting to set
unique_keyword_item_set = set(map(lambda x: x.strip().lower().strip('"'), keyword_item_list))



# Convert the set back to a list if needed
unique_keyword_item_list = list(unique_keyword_item_set)

# Print the length of the list
print("Number of unique items:", len(unique_keyword_item_list))

Number of unique items: 560


In [19]:
# Print the entire list
unique_keyword_item_list

['paprika',
 'new potatoes',
 'iceberg lettuce',
 'lemon zest',
 'homeopathy/remedies',
 'refrigerator',
 'zucchini',
 'baking potatoes',
 'vegan',
 'mahi mahi',
 'boiling water',
 'vinegar',
 'long-grain rice',
 'hard-boiled eggs',
 'romaine lettuce',
 'light corn syrup',
 'lentil',
 'chinese',
 'fresh cilantro',
 'light butter',
 'nutmeg',
 'pumpkin',
 'dry basil',
 'orange roughy',
 'part-skim mozzarella cheese',
 'capers',
 'spaghetti noodles',
 'reduced-sodium chicken broth',
 'cake flour',
 'kosher',
 'southwestern u.s.',
 'rotini pasta',
 'native american',
 'grains',
 'whole wheat flour',
 'roma tomatoes',
 'linguine',
 'fresh sage',
 'long grain rice',
 'citrus',
 'lemon',
 'oregano',
 'chicken breast',
 'spinach',
 'apple cider vinegar',
 'shortening',
 'no cook',
 'strawberry',
 'soy sauce',
 'deep fried',
 'salted butter',
 'lemon pepper',
 'red chile',
 'household cleaner',
 'fresh green beans',
 'turnip',
 'swiss cheese',
 'roast',
 'red wine vinegar',
 'fresh shrimp',
 '

In [20]:
# Iterate through the list and print keywords containing "low", "high", or "quick"
for keyword in unique_keyword_item_list:
    if "vegan" in keyword or "low sodium" in keyword or "low carbs" in keyword or "low cholesterol" in keyword or "high protein" in keyword or "low protein" in keyword:
        print(keyword)

vegan
low protein
very low carbs
low cholesterol
low sodium chicken broth
high protein


In [21]:
# Define the substrings of interest
substrings_of_interest = ["vegan", "low sodium", "low carbs", "low cholesterol", "high protein", "low protein"]

# Initialize a dictionary to store the counts
substring_counts = {substring: 0 for substring in substrings_of_interest}

# Iterate through the list and count keywords containing the specified substrings
for keyword in unique_keyword_item_list:
    for substring in substrings_of_interest:
        if substring in keyword:
            substring_counts[substring] += 1

# Print the counts for each substring
for substring, count in substring_counts.items():
    print(f"Count of '{substring}': {count}")

Count of 'vegan': 1
Count of 'low sodium': 1
Count of 'low carbs': 1
Count of 'low cholesterol': 1
Count of 'high protein': 1
Count of 'low protein': 1


In [22]:
# Define the substrings of interest
substrings_of_interest = ["pizza", "beef", "chicken", "turkey", "pork", "ribs", "rib", "steak", "salmon", "cod", "mahi mahi"]

# Initialize a dictionary to store the counts
substring_counts = {substring: 0 for substring in substrings_of_interest}

# Iterate through the list and count keywords containing the specified substrings
for keyword in unique_keyword_item_list:
    for substring in substrings_of_interest:
        if substring in keyword:
            substring_counts[substring] += 1

# Print the counts for each substring
for substring, count in substring_counts.items():
    print(f"Count of '{substring}': {count}")

Count of 'pizza': 0
Count of 'beef': 7
Count of 'chicken': 13
Count of 'turkey': 3
Count of 'pork': 1
Count of 'ribs': 1
Count of 'rib': 2
Count of 'steak': 1
Count of 'salmon': 3
Count of 'cod': 0
Count of 'mahi mahi': 1


In [23]:
# print("Number of unique keywords:", num_unique_keywords)
# # Print the first 500 unique keywords
# print("Unique keywords:")
# for idx, keyword in enumerate(unique_keywords):
#     if idx < 500:
#         print(keyword)
#     else:
#         break

In [24]:
clean_df.head()


,RecipeId,Name,TotalTime,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,AggregatedRating,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeInstructions,URL
0,429918,Easy Chicken and Biscuits,30M,This is an easy recipe that can be done in 30 ...,haracter0,One Dish Meal,"""< 30 Mins""","""1"", ""1"", ""1"", ""1/4"", ""1/4"", ""4"", ""2"", ""1""","""milk"", ""dried thyme leaves"", ""pepper"", ""ham""",0.0,...,5.3,58.6,1644.3,37.0,1.5,6.5,20.8,5.0,"""In a 3-quart shallow baking dish mix soups, m...",https://www.food.com/recipe/easy-chicken-and-b...
1,498503,Spring Gnocchi With Asparagus and Shrimp,25M,A perfect blend of the heartiest of winter pas...,haracter0,Weeknight,"""< 30 Mins"", ""Easy""","""32 -36"", ""4"", ""1"", ""1"", ""1"", ""2"", ""1"", ""4"", ""...","""butter"", ""lemon, juice and zest of"", ""salt"", ...",0.0,...,6.4,121.2,1018.0,8.5,3.4,2.4,16.3,6.0,"""Boil a large pot of salted water and add gnoc...",https://www.food.com/recipe/spring-gnocchi-wit...
2,263162,Romano Grits,20M,"This recipe can be multiplied by 2, 3, 4. The...",haracter0,Breakfast,"""Very Low Carbs"", ""Low Protein"", ""Low Choleste...","""2"", ""1/2"", ""1/4"", ""1""","""water"", ""yellow corn grits"", ""salt"", ""pecorin...",5.0,...,2.4,14.7,466.0,3.7,0.1,0.1,4.9,2.0,"""Place the water in a medium sauce pan over hi...",https://www.food.com/recipe/romano-grits-263162
3,72878,Lemon Chess Pie,50M,Make and share this Lemon Chess Pie recipe fro...,haracter0,Pie,"""Dessert"", ""< 60 Mins""","""1 1/2"", ""2"", ""4"", ""1"", ""1/2"", ""1""","""sugar"", ""butter"", ""eggs"", ""fresh lemon rind""",4.0,...,4.5,113.4,172.4,48.5,0.9,37.8,4.6,8.0,"""Pre heat oven to 400."", ""Cream together butte...",https://www.food.com/recipe/lemon-chess-pie-72878
4,350007,Slow Cooked Chicken and Dressing,4H25M,This is so delicious! If you want an easy com...,haracter0,One Dish Meal,"""Chicken"", ""Poultry"", ""Meat"", ""Kid Friendly"", ...","""1"", ""6"", ""8"", ""2"", ""2"", ""1"", ""3"", ""4"", ""2"", ""...","""chicken broth"", ""onion"", ""celery ribs"", ""eggs...",0.0,...,11.8,191.3,1275.0,22.5,1.2,3.0,24.8,1.0,"""Combine first 11 ingredients in a large bowl....",https://www.food.com/recipe/slow-cooked-chicke...


In [25]:
# Extract the recipe ingredients from the "RecipeIngredientParts" column
ingredients_series = clean_df['RecipeIngredientParts']

# Split the ingredients into individual items
ingredients_split = ingredients_series.str.split(',')

# Create a list of unique ingredients
unique_ingredients = []
for ingredients in ingredients_split:
    if isinstance(ingredients, list):
        for ingredient in ingredients:
            ingredient = ingredient.strip()
            if ingredient not in unique_ingredients and ingredient != '':
                unique_ingredients.append(ingredient)

In [26]:
# Create a DataFrame with columns representing unique ingredients
ingredient_df = pd.DataFrame(0, index=clean_df.index, columns=unique_ingredients)

# Populate the DataFrame with 1 where ingredient is present in the recipe
for ingredient in unique_ingredients:
    ingredient_df[ingredient] = ingredients_series.str.contains(ingredient, na=False, regex=False).astype(int)


In [27]:
recipes = pd.concat([clean_df, ingredient_df], axis=1)

In [28]:
recipes.head()

,RecipeId,Name,TotalTime,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,AggregatedRating,...,"""low-fat baking mix""","""nonfat vanilla soymilk""","""frozen whole kernel corn""","""cider""","""wine vinegar""","""Roquefort cheese""","""fat-free buttermilk""","""dry thyme leaves""","""panir""","""kasuri methi"""
0,429918,Easy Chicken and Biscuits,30M,This is an easy recipe that can be done in 30 ...,haracter0,One Dish Meal,"""< 30 Mins""","""1"", ""1"", ""1"", ""1/4"", ""1/4"", ""4"", ""2"", ""1""","""milk"", ""dried thyme leaves"", ""pepper"", ""ham""",0.0,...,0,0,0,0,0,0,0,0,0,0
1,498503,Spring Gnocchi With Asparagus and Shrimp,25M,A perfect blend of the heartiest of winter pas...,haracter0,Weeknight,"""< 30 Mins"", ""Easy""","""32 -36"", ""4"", ""1"", ""1"", ""1"", ""2"", ""1"", ""4"", ""...","""butter"", ""lemon, juice and zest of"", ""salt"", ...",0.0,...,0,0,0,0,0,0,0,0,0,0
2,263162,Romano Grits,20M,"This recipe can be multiplied by 2, 3, 4. The...",haracter0,Breakfast,"""Very Low Carbs"", ""Low Protein"", ""Low Choleste...","""2"", ""1/2"", ""1/4"", ""1""","""water"", ""yellow corn grits"", ""salt"", ""pecorin...",5.0,...,0,0,0,0,0,0,0,0,0,0
3,72878,Lemon Chess Pie,50M,Make and share this Lemon Chess Pie recipe fro...,haracter0,Pie,"""Dessert"", ""< 60 Mins""","""1 1/2"", ""2"", ""4"", ""1"", ""1/2"", ""1""","""sugar"", ""butter"", ""eggs"", ""fresh lemon rind""",4.0,...,0,0,0,0,0,0,0,0,0,0
4,350007,Slow Cooked Chicken and Dressing,4H25M,This is so delicious! If you want an easy com...,haracter0,One Dish Meal,"""Chicken"", ""Poultry"", ""Meat"", ""Kid Friendly"", ...","""1"", ""6"", ""8"", ""2"", ""2"", ""1"", ""3"", ""4"", ""2"", ""...","""chicken broth"", ""onion"", ""celery ribs"", ""eggs...",0.0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
# Melt the DataFrame to convert ingredient columns into rows
melted_df = recipes.melt(var_name='Ingredient', value_name='Present')

# Filter out rows where ingredient is present
present_ingredients = melted_df[melted_df['Present'] == 1]

# Count the occurrences of each ingredient
ingredient_counts = present_ingredients['Ingredient'].value_counts().reset_index()

# Rename columns
ingredient_counts.columns = ['Ingredient', 'Count']

In [30]:
# Define the thresholds for each category
category_thresholds = {
    'low sodium': 140,    # Daily recommended intake for low sodium
    'low calorie': 200,   # Arbitrary threshold for low calorie
    'high protein': 20,   # Arbitrary threshold for high protein
    'low fat': 3,         # Arbitrary threshold for low fat
    'high fat': 20,       # Arbitrary threshold for high fat
    'high fiber': 5,      # Arbitrary threshold for high fiber
    'low carb': 20,       # Arbitrary threshold for low carb
    'low cholesterol': 20, # Arbitrary threshold for low cholesterol
    'low sugar': 5        # Arbitrary threshold for low sugar
}

In [31]:
# Create new columns for each category and assign boolean values based on thresholds
for category, threshold in category_thresholds.items():
    recipes[category] = False
    if category == 'low sodium':
        recipes.loc[recipes['SodiumContent'] < threshold, category] = True
    elif category == 'low calorie':
        recipes.loc[recipes['Calories'] < threshold, category] = True
    elif category == 'high protein':
        recipes.loc[recipes['ProteinContent'] > threshold, category] = True
    elif category == 'low fat':
        recipes.loc[recipes['FatContent'] < threshold, category] = True
    elif category == 'high fat':
        recipes.loc[recipes['FatContent'] > threshold, category] = True
    elif category == 'high fiber':
        recipes.loc[recipes['FiberContent'] > threshold, category] = True
    elif category == 'low carb':
        recipes.loc[recipes['CarbohydrateContent'] < threshold, category] = True
    elif category == 'low cholesterol':
        recipes.loc[recipes['CholesterolContent'] < threshold, category] = True
    elif category == 'low sugar':
        recipes.loc[recipes['SugarContent'] < threshold, category] = True

In [32]:
# Drop all columns except the newly created category columns
columns_to_keep = ["Name"] + list(category_thresholds.keys())
recipes2 = recipes[columns_to_keep]

In [33]:
# Function to determine the category for each recipe
def determine_category(row):
    for category, threshold in category_thresholds.items():
        if category == 'low sodium':
            if row['SodiumContent'] < threshold:
                return category
        elif category == 'low calorie':
            if row['Calories'] < threshold:
                return category
        elif category == 'high protein':
            if row['ProteinContent'] > threshold:
                return category
        elif category == 'low fat':
            if row['FatContent'] < threshold:
                return category
        elif category == 'high fat':
            if row['FatContent'] > threshold:
                return category
        elif category == 'high fiber':
            if row['FiberContent'] > threshold:
                return category
        elif category == 'low carb':
            if row['CarbohydrateContent'] < threshold:
                return category
        elif category == 'low cholesterol':
            if row['CholesterolContent'] < threshold:
                return category
        elif category == 'low sugar':
            if row['SugarContent'] < threshold:
                return category
    # If no category is assigned, return None
    return None

# Apply the determine_category function to each row to determine the category for each recipe
recipes['AssignedCategory'] = recipes.apply(determine_category, axis=1)

In [34]:
recipes.head()

,RecipeId,Name,TotalTime,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,AggregatedRating,...,low sodium,low calorie,high protein,low fat,high fat,high fiber,low carb,low cholesterol,low sugar,AssignedCategory
0,429918,Easy Chicken and Biscuits,30M,This is an easy recipe that can be done in 30 ...,haracter0,One Dish Meal,"""< 30 Mins""","""1"", ""1"", ""1"", ""1/4"", ""1/4"", ""4"", ""2"", ""1""","""milk"", ""dried thyme leaves"", ""pepper"", ""ham""",0.0,...,False,False,True,False,False,False,False,False,False,high protein
1,498503,Spring Gnocchi With Asparagus and Shrimp,25M,A perfect blend of the heartiest of winter pas...,haracter0,Weeknight,"""< 30 Mins"", ""Easy""","""32 -36"", ""4"", ""1"", ""1"", ""1"", ""2"", ""1"", ""4"", ""...","""butter"", ""lemon, juice and zest of"", ""salt"", ...",0.0,...,False,True,False,False,False,False,True,False,True,low calorie
2,263162,Romano Grits,20M,"This recipe can be multiplied by 2, 3, 4. The...",haracter0,Breakfast,"""Very Low Carbs"", ""Low Protein"", ""Low Choleste...","""2"", ""1/2"", ""1/4"", ""1""","""water"", ""yellow corn grits"", ""salt"", ""pecorin...",5.0,...,False,True,False,False,False,False,True,True,True,low calorie
3,72878,Lemon Chess Pie,50M,Make and share this Lemon Chess Pie recipe fro...,haracter0,Pie,"""Dessert"", ""< 60 Mins""","""1 1/2"", ""2"", ""4"", ""1"", ""1/2"", ""1""","""sugar"", ""butter"", ""eggs"", ""fresh lemon rind""",4.0,...,False,False,False,False,False,False,False,False,False,None
4,350007,Slow Cooked Chicken and Dressing,4H25M,This is so delicious! If you want an easy com...,haracter0,One Dish Meal,"""Chicken"", ""Poultry"", ""Meat"", ""Kid Friendly"", ...","""1"", ""6"", ""8"", ""2"", ""2"", ""1"", ""3"", ""4"", ""2"", ""...","""chicken broth"", ""onion"", ""celery ribs"", ""eggs...",0.0,...,False,False,True,False,True,False,False,False,True,high protein


# Drop all columns except 'Name' and 'AssignedCategory'
columns_to_keep2 = ['Name', 'AssignedCategory', 'SugarContent', 'CholesterolContent', 'CarbohydrateContent', 'FiberContent', 'FatContent', 'ProteinContent', 'Calories', 'SodiumContent']
recipes = recipes[columns_to_keep2]
recipes.head()

In [35]:
# Adding Health Keywords to Keywords column
# Define the criteria for adding keywords
criteria = (
    recipes['FiberContent'] > 5
) | (
    recipes['RecipeCategory'].str.contains('High In', na=False)
) | (
    recipes['ProteinContent'] > 20
) | (
    recipes['RecipeCategory'].str.contains('Homeopathy/Remedies', na=False)
) | (
    recipes['RecipeCategory'].str.contains('Kid Friendly', na=False)
) | (
    recipes['RecipeCategory'].str.contains('Lactose Free', na=False)
) | (
    recipes['RecipeCategory'].str.contains('Low Cholesterol', na=False)
) | (
    recipes['RecipeCategory'].str.contains('Low Protein', na=False)
) | (
    recipes['RecipeCategory'].str.contains('No Shell Fish', na=False)
) | (
    recipes['RecipeCategory'].str.contains('Soy/Tofu', na=False)
) | (
    recipes['CarbohydrateContent'] < 5
) | (
    recipes['RecipeCategory'].str.contains('High In.*Diabetic Friendly', na=False)
) | (
    recipes['RecipeCategory'].str.contains('Bath/Beauty', na=False)
) | (
    recipes['RecipeCategory'].str.contains('Egg Free', na=False)
) | (
    recipes['RecipeCategory'].str.contains('Healthy', na=False)
) | (
    recipes['RecipeCategory'].str.contains('Kosher', na=False)
) | (
    recipes['RecipeCategory'].str.contains('Toddler Friendly', na=False)
) | (
    recipes['RecipeCategory'].str.contains('Vegan', na=False)
) | (
    recipes['RecipeCategory'].str.contains('Vegetable', na=False)
)

In [36]:
# Add the specified keywords to the Keywords column for recipes that meet the criteria
clean_df.loc[criteria, 'Keywords'] = recipes.loc[criteria, 'Keywords'] + ', "High Fiber", "High Protein", "Homeopathy/Remedies", "Kid Friendly", "Lactose Free", "Low Cholesterol", "Low Protein", "No Shell Fish", "Soy/Tofu", "Very Low Carbs", "Diabetic Friendly", "Bath/Beauty", "Egg Free", "Healthy", "Kosher", "Toddler Friendly", "Vegan", "Vegetable"'

In [37]:
# Save clean data to a new JSON file
clean_df.to_json("data/clean/cleanSamplerecipes.json", orient="records")

In [39]:
# Save clean data to a new CSV file
clean_df.to_csv("data/clean/cleanSamplerecipes.csv", index=False)

# Importing necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Define features (X) and target variable (y)
X = clean_df[['RecipeServings', 'AggregatedRating', 'ReviewCount']]  # Features
y = clean_df['RecipeCategory']  # Target variable

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initializing and training the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# Making predictions on the test set
y_pred = rf_classifier.predict(X_test)

# Calculating the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of Random Forest classifier:", accuracy)